# Build an Agent

语言模型本身无法执行操作 - 它们只是输出文本。 `LangChain` 的一个重要用例是创建 agents。Agents 是使用 LLM 作为推理引擎的系统，以确定要采取哪些操作以及执行操作所需的输入。执行操作后，可以将结果反馈回 LLM，以确定是否需要更多操作，或者是否可以完成。这通常通过工具调用实现。

## End-to-end agent

下面的代码片段表示一个功能齐全的 Agent，它使用 LLM 来决定使用哪些工具。它配备了一个通用搜索工具。它具有对话记忆 - 意味着它可以作为多轮聊天机器人使用。

In [14]:
# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Import relevant functionality
from langchain_openai import ChatOpenAI
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langchain.prompts import PromptTemplate
from langgraph.prebuilt import create_react_agent


In [15]:
# Create the agent

# MemorySaver
memory = MemorySaver()

# Chat model
model = ChatOpenAI(model="gpt-4o-mini", temperature=0, max_tokens=1000)

# Tool
search = TavilySearch(max_results=2)
tools = [search]

# Create the agent (using model, tools, and memory storage)
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [16]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}

input_message = {
    "role": "user",
    "content": "Hi, I'm Benwu and I live in Zhuhai, Guangdong Province, China.",
}

# Stream the agent's response
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Benwu and I live in Zhuhai, Guangdong Province, China.
================================== Ai Message ==================================

Hello Benwu! It's great to meet you. How can I assist you today?


In [17]:
input_message = {
    "role": "user",
    "content": "What's the weather where I live?",
}

for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's the weather where I live?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_5G1ZHd55bHlrk8PpREVG2qep)
 Call ID: call_5G1ZHd55bHlrk8PpREVG2qep
  Args:
    query: Zhuhai Guangdong weather
    topic: general
================================= Tool Message =================================
Name: tavily_search

{"query": "Zhuhai Guangdong weather", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in Zhuhai, Guangdong", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Zhuhai', 'region': 'Guangdong', 'country': 'China', 'lat': 22.2769, 'lon': 113.5678, 'tz_id': 'Asia/Shanghai', 'localtime_epoch': 1751956328, 'localtime': '2025-07-08 14:32'}, 'current': {'last_updated_epoch': 1751956200, 'last_updated': '2025-07-08 14:30', 'temp_c': 32.2, 'temp_f': 90.0, 'is_day': 1, 'condition':

## Define tools

使用Tavily（搜索引擎）作为工具。获取API_KEY并加入环境变量中。


In [18]:
# Test of the Tavily search tool
from langchain_tavily import TavilySearch

search = TavilySearch(max_results=2)
search_results = search.invoke("What is the weather in Zhuhai, Guangdong Province, China?")
print(search_results)

# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

{'query': 'What is the weather in Zhuhai, Guangdong Province, China?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in Zhuhai, Guangdong Province, China', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Zhuhai', 'region': 'Guangdong', 'country': 'China', 'lat': 22.2769, 'lon': 113.5678, 'tz_id': 'Asia/Shanghai', 'localtime_epoch': 1751957327, 'localtime': '2025-07-08 14:48'}, 'current': {'last_updated_epoch': 1751957100, 'last_updated': '2025-07-08 14:45', 'temp_c': 32.2, 'temp_f': 90.0, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 13.6, 'wind_kph': 22.0, 'wind_degree': 204, 'wind_dir': 'SSW', 'pressure_mb': 1002.0, 'pressure_in': 29.59, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 71, 'cloud': 50, 'feelslike_c': 40.5, 'feelslike_f': 104.9, 'windchill_c': 31.5, 'windchill_f': 88.6, 'heatindex_c': 38.5, 'heatindex_f': 101.3, 'dew

通过定义python函数等方式可以自定义工具。 `LangChain` 支持使用自定义工具。



## Using Language Models

`LangChain` 支持不同的语言模型。


In [19]:
from dotenv import load_dotenv
load_dotenv()

True

In [20]:
# Create the model

from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

model = ChatOpenAI(model="gpt-4o-mini", temperature=0, max_tokens=1000)

可以通过传入消息列表来调用语言模型。 默认情况下，响应是一个 `content` 字符串。

In [21]:
query = "Hi!"
response = model.invoke([{"role": "user", "content": query}])
response.text()

'Hello! How can I assist you today?'

可以看到启用此模型进行工具调用是什么样的。 为了启用它，我们使用 `.bind_tools` 为语言模型提供这些工具的知识

In [22]:
model_with_tools = model.bind_tools(tools)

启动模型并查看 `content` 字段以及 `tool_calls` 字段，获取相应和模型对工具的调用（实际上没有调用任何工具，只是模型的请求）。

In [23]:
query = "Hi!"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: Hello! How can I assist you today?

Tool calls: []


可以看到，对于简单的输入，模型并不需要调用任何工具。

现在尝试输入一些可能需要调用工具的内容。

In [24]:
query = "Search for the weather in Zhuhai, Guangdong Province, China."
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 

Tool calls: [{'name': 'tavily_search', 'args': {'query': 'weather in Zhuhai, Guangdong Province, China', 'topic': 'general'}, 'id': 'call_7zEzSwNHMXdmBkD3J5KrrWCR', 'type': 'tool_call'}]


我们可以看到现在没有文本内容，但有一个工具调用！ 它希望我们调用 Tavily 搜索工具。

这还没有调用该工具 - 它只是告诉我们这样做。 为了实际调用它，我们将需要创建我们的 Agent。

这告诉我们模型本身并不直接调用工具，只是发出请求。实际调用工具的是 Agent。

## Create the agent

现在已经定义了工具和 LLM，就可以创建 Agent 了。 我们将使用 LangGraph 来构建 Agent。 

目前，我们正在使用高级接口来构建 Agent，但 LangGraph 的好处在于，这个高级接口由一个低级、高度可控的 API 支持，以防我们想要修改 Agent 逻辑。

使用 LLM 和工具初始化 Agent。

注意传入的是 `model`，而不是 `model_with_tools`。 这是因为 `create_react_agent` 会在后台调用 `.bind_tools`。

In [63]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

## Run the agent

现在可以使用一些查询来运行 Agent 了！ 请注意，目前，这些都是*无状态查询*（它不会记住之前的交互）。 

Agent 将在交互结束时返回*最终*状态（包括任何输入）。

首先让我们看看在不需要调用工具时它是如何响应的:

In [67]:
input_message = {"role": "user", "content": "Hi!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

response["messages"]

================================ Human Message =================================

Hi!
================================== Ai Message ==================================

Hello! How can I assist you today?


[HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}, id='2c9c2a08-b818-42f3-b6e3-e69c54aeccf8'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 792, 'total_tokens': 802, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bqx1tyjIQe5p52yBGD2ebboStlbUU', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c287da67-842f-491a-842a-e5abf3deddca-0', usage_metadata={'input_tokens': 792, 'output_tokens': 10, 'total_tokens': 802, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

接着看看在需要调用工具时是如何响应的，通过 `.invoke` 调用Agent:

In [68]:
input_message = {"role": "user", "content": "Search for the weather in Zhuhai, Guangdong Province, China."}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Search for the weather in Zhuhai, Guangdong Province, China.
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_0Yq53UMjeekHin4Dl19SYy0V)
 Call ID: call_0Yq53UMjeekHin4Dl19SYy0V
  Args:
    query: weather in Zhuhai, Guangdong Province, China
    topic: general
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in Zhuhai, Guangdong Province, China", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in Zhuhai, Guangdong Province, China", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Zhuhai', 'region': 'Guangdong', 'country': 'China', 'lat': 22.2769, 'lon': 113.5678, 'tz_id': 'Asia/Shanghai', 'localtime_epoch': 1751957327, 'localtime': '2025-07-08 14:48'}, 'current': {'last_updated_epoch': 1751957100, 'last_upd

## Streaming Messages

Agents需要多步执行时，通过查看流信息（流式传输信息）显示中间进程。

In [29]:
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Search for the weather in Zhuhai, Guangdong Province, China.
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_cEugRpP1OQ3CqSFAy7LHI50r)
 Call ID: call_cEugRpP1OQ3CqSFAy7LHI50r
  Args:
    query: weather in Zhuhai, Guangdong Province, China
    topic: general
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in Zhuhai, Guangdong Province, China", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in Zhuhai, Guangdong Province, China", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Zhuhai', 'region': 'Guangdong', 'country': 'China', 'lat': 22.2769, 'lon': 113.5678, 'tz_id': 'Asia/Shanghai', 'localtime_epoch': 1751957327, 'localtime': '2025-07-08 14:48'}, 'current': {'last_updated_epoch': 1751957100, 'last_upd

## Streaming tokens

除了流式传输消息之外，流式传输tokes也很有用。可以通过指定 `stream_mode="messages"` 来做到这一点。

In [30]:
for step, metadata in agent_executor.stream(
    {"messages": [input_message]}, stream_mode="messages"
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The| current| weather| in| Zh|uh|ai|,| Guangdong| Province|,| China| is| as| follows|:

|-| **|Temperature|**|:| |32|.|2|°C| (|90|.|0|°F|)
|-| **|Condition|**|:| Over|cast|
|-| **|Wind|**|:| |13|.|6| mph| (|22|.|0| k|ph|)| from| the| S|SW|
|-| **|Humidity|**|:| |71|%
|-| **|Pressure|**|:| |100|2|.|0| mb|
|-| **|Visibility|**|:| |10| km|
|-| **|UV| Index|**|:| |9|.|7|

|For| more| detailed| information|,| you| can| visit| [|Weather|API|](|https|://|www|.weather|api|.com|/|).| 

|Additionally|,| if| you're| interested| in| the| weather| forecast| for| August| |202|5|,| you| can| check| [|Trip|venue|](|https|://|trip|venue|.com|/weather|/ch|ina|/l|179|043|7|/|zh|uh|ai|/|aug|ust|).|

## Adding in memory
如前所述此Agent是无状态的。这意味着它不记得之前的交互。

In [69]:
input_message = {"role": "user", "content": "Hi, I'm Benwu!"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Benwu!
================================== Ai Message ==================================

Hello Benwu! How can I assist you today?
================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

I'm sorry, but I don't have access to personal information about users unless it has been shared with me in the course of our conversation. If you'd like to tell me your name, I can remember it for the duration of our chat!


为了实现记忆功能，我们需要传入一个检查点 `checkpoint` 和一个 `thread_id`（以便它知道要从哪个线程/对话恢复）。

In [70]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [71]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [72]:
input_message = {"role": "user", "content": "Hi, I'm Benwu!"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Benwu!
================================== Ai Message ==================================

Hello Benwu! How can I assist you today?


In [73]:
input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

Your name is Benwu! How can I help you today?


想要创建一个新的聊天只需要创建一个新的 `thread_id` （更改进程名）。

In [74]:
# highlight-next-line
config = {"configurable": {"thread_id": "xyz123"}}

input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

I don't have access to personal data about users unless it has been shared with me in the course of our conversation. Therefore, I don't know your name. If you'd like to share it, feel free!


## LangGraph

上述Agent仅仅使用了一个搜索tool，那如果我有很多个tool，需要他们之间按顺序进行协作，完成指定任务呢？

`LangGraph` 提供了这样的框架：将每个工具定义为有向图中的结点（node），通过有向边（edge）进行连接表示执行的流程，用图（Graph）的方式表示Agent的工作流程。每个结点包含一个状态，所有结点函数接收并返回这个结构。

下面通过一个文本分析总结Agent进行学习：

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()
from typing import TypedDict, List
from langgraph.graph import StateGraph, END
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

llm = ChatOpenAI(model = 'gpt-4o-mini', temperature = 0)

`LangGraph` 是基于“有状态流程图”（Stateful Graph）的系统，每个结点处理一个任务，同时接收和传递一个“state”字典。我们需要为Agent定义状态，使用TypedDict定义状态结构。

`TypedDict` 是 Python `typing` 模块中的一种类型注解工具，允许你为字典（`dict`）指定键（key）和值（value）的类型。它让你可以像定义一个类一样，定义结构化的字典，并且在静态类型检查工具（如 mypy、Pyright）中获得类型支持。

In [2]:
# TypedDict：用于定义状态字典的类型，包含文本、分类、实体和摘要等字段，能够对键值类型进行严格检查
class State(TypedDict):  
	text: str             # 存储原始输入文本    
	classification: str   # 存储分类结果（例如类别）    
	entities: List[str]   # 存储提取出的实体列表（例如命名实体）    
	summary: str          # 存储文本的简洁摘要


定义工具，`LangGraph` 接受Python自定义函数作为工具：

In [5]:
def classification_node(state: State) -> State:
	"""   
	将文本分类为预定义的类别之一。   
	参数:    
		 state (State): 当前状态字典，包含待分类的文本   
	返回:  
			 dict: 返回包含 "classification" 键的字典，其值为分类结果   
	类别:  
			 - News: 新闻报道类       
			 - Blog: 个人或非正式写作       
			 - Research: 学术或科研内容      
			 - Other: 不属于以上类别的其它内容
	"""
	# 定义 prompt 模板，指示模型对给定文本进行分类   
	prompt = PromptTemplate(input_variables=["text"], template="Classify the following text into one of the categories: News, Blog, Research, Other. Text: {text}") 
	
	# 根据当前 state 中的文本格式化 prompt  
	message = HumanMessage(content=prompt.format(text=state["text"])) 
	# 调用语言模型对文本进行分类  
	classification = llm.invoke([message]).content.strip()   
	print("Do Classification.")
	# 返回包含分类结果的字典  
	return {"classification": classification}


def entity_extraction_node(state: State) -> State: 
	# 用于从文本中识别并提取命名实体（按人名、组织、位置分类）  
	# 创建实体提取 prompt 模板，指定需要查找的实体和格式（逗号分隔） 
	prompt = PromptTemplate( 
		input_variables=["text"],      
		template="Extract all the entities (Person, Organization, Location) from the following text. Text: {text}" 
	) 
	# 根据 state 中的文本格式化 prompt 并包装为 HumanMessage  
	message = HumanMessage(content=prompt.format(text=state["text"]))  
	# 发送给语言模型，获取响应，清除空白后按逗号分隔拆分为列表  
	entities = llm.invoke([message]).content.strip().split(", ") 
	print("Extract Entities.")
	# 返回包含实体列表的字典，将与 agent state 合并  
	return {"entities": entities}

def summarize_node(state: State) -> State:  
	# 创建摘要 prompt 模板，指示模型用一句话对文本进行总结    
	summarization_prompt = PromptTemplate.from_template(   
		"""Summarize the following text in one short sentence.       
		Text: {text}        
		Summary:"""   
	)   
	# 利用 "|" 运算符将 prompt 模板和语言模型连接起来形成一个 chain   
	# 运算符重载允许将 prompt 和模型组合成一个处理链
	# 这里的 chain 将 prompt 和模型结合起来，形成一个处理流程 
	chain = summarization_prompt | llm    
	# 执行 chain，将文本传递给模型进行摘要    
	response = chain.invoke({"text": state["text"]}) 
	print(f"Generated Summary\n")   
	# 返回包含摘要的字典，将合并进 agent state   
	return {"summary": response.content}

接着就是核心部分：构建 `StateGraph` 流程图，主要包含以下内容：

每个函数作为一个结点，接受并更新状态，将状态传递给下一个结点函数：

In [6]:
# 创建有状态流程图
workflow = StateGraph(State)

# 向图中添加各个节点
workflow.add_node("classification_node", classification_node)
workflow.add_node("entity_extraction", entity_extraction_node)
workflow.add_node("summarization", summarize_node)

将结点用有向边进行连接，表示状态传递方向和工作顺序：

In [ ]:
# 定义图中各节点之间的边关系
workflow.set_entry_point("classification_node") # 定义入口结点

# 将结点用有向边进行连接，表示状态传递方向和工作顺序
workflow.add_edge("classification_node", "entity_extraction")
workflow.add_edge("entity_extraction", "summarization")

workflow.add_edge("summarization", END) # 定义结束结点

最后编译，返回结果是根据用 `LangGraph` 构建的步骤图编译生成的可以 `invoke()` 或 `astream()` 执行的 `CompileGraph` 对象：

In [10]:
agent = workflow.compile()

初始状态仅为用户输入文本。每个结点函数都会返回一个字典，包含本节点新生成的内容（即本结点完成的任务）。

通过 `LangGraph` 中的 `StateGraph` ，自动将每个回的字典与当前的 state 合并，形成新的 state，传递给下一个节点，形成有状态流程图的工作流。

尝试运行：

In [12]:
# 一段关于 LangGraph agent 的样本文本
sample_text = """
A LangGraph agent is a structured, stateful reasoning system built using LangGraph, 
a library designed for defining and executing dynamic workflows with language models. 
Unlike traditional agents that rely on linear or reactive chains, a LangGraph agent 
operates over a directed graph where each node represents a step—such as calling a tool, 
processing a response, or checking a condition—and edges define possible transitions based 
on the current state. The agent maintains a shared state throughout execution, often 
defined using TypedDict, allowing for consistent data flow and decision-making across 
steps. This architecture enables complex, controllable, and interpretable behavior in 
multi-step language model applications such as tool-augmented question answering, autonomous 
reasoning, or multi-turn dialogue management.
"""

# 构造包含样本文本的初始 state
state_input = {"text": sample_text}
# 运行 agent 的全流程
result = agent.invoke(state_input)

# 输出各个部分的结果：
# 分类结果（News、Blog、Research 或 Other）
print("Classification:", result["classification"])

# 提取出的实体（人名、组织、地点等）
print("\nEntities:")
for entity in result["entities"]:
	print(entity)

# 生成的文本摘要
print("\nSummary:", result["summary"])

Do Classification.
Extract Entities.
Generated Summary

Classification: The text can be classified as **Research**.

Entities:
From the provided text
the following entities can be extracted:

**Organizations:**
- LangGraph

**Locations:**
- None identified in the text.

**Persons:**
- None identified in the text. 

The text primarily discusses a technology (LangGraph) and its functionalities without mentioning specific individuals or locations.

Summary: A LangGraph agent is a stateful reasoning system that utilizes a directed graph structure for dynamic workflows with language models, enabling complex and interpretable multi-step applications.
